In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import itertools
import pickle
import sys
# sys.path.append("..")
from utils import data_proc_tools as dpt
from utils import plot_tools as pt
from utils.custom_metrics import recall, precision, binary_accuracy
from utils.custom_metrics import recall_np, precision_np, binary_accuracy_np, multilabel_confusion_matrix
from utils.text_sum_models import Seq2Seq
import random
random.seed(42)
random_state=1000
pd.set_option('display.max_colwidth', -1)
import pylab

from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, SeparableConv1D
from keras.layers import Flatten, Dropout, Input, LSTM, BatchNormalization, Activation, TimeDistributed
from keras.layers.embeddings import Embedding

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

Using TensorFlow backend.


In [3]:
dir = '/vol/medic02/users/ag6516/radiology_report_summarisation/'
data_dir = dir + 'data/'

aug = 'aug'

model_output_dir = dir + 'trained_models/seq2seq/'

train_df = pd.read_pickle(data_dir + 'train/{}_train.pkl'.format(aug))
val_df = pd.read_pickle(data_dir + 'val/val.pkl')

## Load and prepare sequence data for training

In [4]:
train_df.head()

,examid,report,all_mesh,single_mesh
0,CXR1000_IM-0003,"[increased, opacity, within, right, upper, lobe, possible, mass, associated, area, atelectasis, focal, consolidation, ., cardiac, silhouette, within, normal, limits, ., opacity, left, midlung, overlying, posterior, left, 5th, rib, may, represent, focal, airspace, disease, ., increased, opacity, right, upper, lobe, associated, atelectasis, may, represent, focal, consolidation, mass, lesion, atelectasis, ., recommend, chest, ct, evaluation, ., opacity, overlying, left, 5th, rib, may, represent, focal, airspace, disease]","[opacity, lung, lingula, opacity, lung, upper_lobe, right, pulmonary_atelectasis, upper_lobe, right]","[opacity, lung, upper_lobe, right]"
1,CXR1001_IM-0004,"[interstitial, markings, diffusely, prominent, throughout, lungs, ., heart, size, normal, ., pulmonary, normal, ., diffuse, fibrosis]","[diffuse, markings, lung, bilateral, interstitial, diffuse, prominent]","[markings, lung, bilateral, interstitial, diffuse, prominent]"
2,CXR1002_IM-0004,"[status, post, left, mastectomy, ., heart, size, normal, ., lungs, clear]",[left],[left]
3,CXR1003_IM-0005,"[heart, size, pulmonary, vascularity, appear, within, normal, limits, ., retrocardiac, soft, tissue, density, present, ., appears, air, within, suggest, represents, hiatal, hernia, ., vascular, calcification, noted, ., calcified, granuloma, seen, ., interval, development, bandlike, opacity, left, lung, base, ., may, represent, atelectasis, ., osteopenia, present, spine, ., retrocardiac, soft, tissue, density, ., appearance, suggests, hiatal, hernia, ., left, base, bandlike, opacity, ., appearance, suggests, atelectasis]","[bone_diseases_metabolic, spine, calcified_granuloma, calcinosis, blood_vessels, density, retrocardiac, opacity, lung, base, left]","[opacity, lung, base, left]"
4,CXR1004_IM-0005,"[heart, ,, pulmonary, mediastinum, within, normal, limits, ., aorta, tortuous, ectatic, ., degenerative, changes, acromioclavicular, joints, ., degenerative, changes, spine, ., ivc, identified]","[aorta, tortuous, catheters_indwelling, shoulder, bilateral, degenerative, spine, degenerative]","[shoulder, bilateral, degenerative]"


In [5]:
# prepend and append start and end tokens to mesh captions and text reports
start_token = 'start'
end_token = 'end'
unknown_token = '**unknown**'
max_mesh_length = 13 # avg. + 1std. + start + end
max_report_length = 37 # avg. + 1std. + start + end

train_df['pad_mesh_caption'] = train_df.all_mesh.apply(lambda x: dpt.pad_sequence(x, max_mesh_length, start_token, end_token))
train_df['pad_text_report'] = train_df.report.apply(lambda x: dpt.pad_sequence(x, max_report_length, start_token, end_token))

val_df['pad_mesh_caption'] = val_df.all_mesh.apply(lambda x: dpt.pad_sequence(x, max_mesh_length, start_token, end_token))
val_df['pad_text_report'] = val_df.report.apply(lambda x: dpt.pad_sequence(x, max_report_length, start_token, end_token))

Sequence empty
Sequence empty
Sequence empty
Sequence empty
Sequence empty
Sequence empty
Sequence empty


## Vectorise text reports and mesh captions

In [6]:
train_mesh = list(train_df.pad_mesh_caption)
train_reports = list(train_df.pad_text_report)

# vectorize mesh captions
dpt.mesh_to_vectors(train_mesh, dicts_dir=data_dir+'dicts/', load_dicts=True, save=True, output_dir=data_dir+'train/')

# vectorise reports
dpt.reports_to_vectors(train_reports, dicts_dir=data_dir+'dicts/', load_dicts=True, save=True, output_dir=data_dir+'train/')

Creating list of mesh ids from loaded dictionaries


In [7]:
val_reports = list(val_df.pad_text_report)
val_mesh = list(val_df.pad_mesh_caption)

# vectorise val reports + mesh using the same dict as created for train
dpt.mesh_to_vectors(val_mesh, dicts_dir=data_dir+'dicts/', load_dicts=True, save=True, output_dir=data_dir+'val/')
dpt.reports_to_vectors(val_reports, dicts_dir=data_dir+'dicts/', load_dicts=True, save=True, output_dir=data_dir+'val/')

Creating list of mesh ids from loaded dictionaries


In [8]:
word_to_id, id_to_word = dpt.load_report_dicts(data_dir+'dicts/')
mesh_to_id, id_to_mesh = dpt.load_mesh_dicts(data_dir+'dicts/')

report_vocab_length = len(word_to_id)
mesh_vocab_length = len(mesh_to_id)

In [9]:
report_vocab_length, mesh_vocab_length

(1475, 128)

In [10]:
# Create arrays of indixes for input sentences, output entities and shifted output entities (t-1)
train_token_ids_array = np.load(data_dir + 'train/token_ids_array.npy')
train_mesh_ids_array = np.load(data_dir + 'train/mesh_ids_array.npy')
train_mesh_ids_array_shifted =[np.concatenate((mesh_to_id[start_token], t[:-1]), axis=None) for t in train_mesh_ids_array]
train_mesh_ids_array_shifted = np.asarray(train_mesh_ids_array_shifted)

val_token_ids_array = np.load(data_dir + 'val/token_ids_array.npy')
val_mesh_ids_array = np.load(data_dir + 'val/mesh_ids_array.npy')
val_mesh_ids_array_shifted = [np.concatenate((mesh_to_id[start_token], t[:-1]), axis=None) for t in val_mesh_ids_array]
val_mesh_ids_array_shifted = np.asarray(val_mesh_ids_array_shifted)

In [11]:
train_token_ids_array[0]

array([1448,  568, 1409,  420, 1109,  563,  501, 1166, 1092,  380, 1363,
       1472,   50,  197,  227, 1400,  191,  420,  955,  238,  227, 1409,
        117,  323, 1264,  121,  117, 1249,  232, 1157, 1300,   50,  362,
         71,  227,  568,  917])

In [12]:
print([id_to_word[idx] for idx in train_token_ids_array[0]])

['start', 'increased', 'opacity', 'within', 'right', 'upper', 'lobe', 'possible', 'mass', 'associated', 'area', 'atelectasis', 'focal', 'consolidation', '.', 'cardiac', 'silhouette', 'within', 'normal', 'limits', '.', 'opacity', 'left', 'midlung', 'overlying', 'posterior', 'left', '5th', 'rib', 'may', 'represent', 'focal', 'airspace', 'disease', '.', 'increased', 'end']


In [13]:
print([id_to_mesh[idx] for idx in train_mesh_ids_array[0]])

['start', 'opacity', 'lung', 'lingula', 'opacity', 'lung', 'upper_lobe', 'right', 'pulmonary_atelectasis', 'upper_lobe', 'right', 'end', 'end']


In [14]:
print([id_to_mesh[idx] for idx in train_mesh_ids_array_shifted[0]])

['start', 'start', 'opacity', 'lung', 'lingula', 'opacity', 'lung', 'upper_lobe', 'right', 'pulmonary_atelectasis', 'upper_lobe', 'right', 'end']


In [15]:
# one-hot-encode
one_hot_reports_train = dpt.one_hot_sequence(train_token_ids_array, report_vocab_length)
one_hot_mesh_train = dpt.one_hot_sequence(train_mesh_ids_array, mesh_vocab_length)
one_hot_mesh_shifted_train = dpt.one_hot_sequence(train_mesh_ids_array_shifted, mesh_vocab_length)

one_hot_reports_val = dpt.one_hot_sequence(val_token_ids_array, report_vocab_length)
one_hot_mesh_val = dpt.one_hot_sequence(val_mesh_ids_array, mesh_vocab_length)
one_hot_mesh_shifted_val = dpt.one_hot_sequence(val_mesh_ids_array_shifted, mesh_vocab_length)

In [16]:
one_hot_reports_train.shape, one_hot_mesh_train.shape, one_hot_mesh_shifted_train.shape

((5148, 37, 1475), (5148, 13, 128), (5148, 13, 128))

## Train Seq-to-Seq Model

In [19]:
input_dim = len(word_to_id)
output_dim = len(mesh_to_id)
hidden_dim = 512
input_seq_length = max_report_length
output_seq_length = max_mesh_length
epochs = 50
optimizer = 'adam'
loss='categorical_crossentropy'
batch_size = 128

new_experiment = Seq2Seq(epochs=epochs,
                               metrics=['accuracy', binary_accuracy,recall,precision],
                               optimizer=optimizer,
                               loss=loss,
                               batch_size=batch_size, 
                               input_dim=input_dim,
                               output_dim=output_dim,
                               hidden_dim=hidden_dim,
                               input_seq_length=input_seq_length,
                               output_seq_length=output_seq_length,
                               verbose=True)
new_experiment.build_model()
new_experiment.model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(128, 37, 1475)]    0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(128, 512), (128, 5 4071424     input_7[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LSTM)                   multiple             1312768     input_8[0][0]                    
                                                                 lstm_4[0][1]               

In [ ]:
# create batch generators
# train_batch_generator = dpt.batch_generator_seq2seq(train_token_ids_array, report_vocab_length, train_mesh_ids_array, 
#                                                    train_mesh_ids_array_shifted, mesh_vocab_length, batch_size)

# val_batch_generator = dpt.batch_generator_seq2seq(val_token_ids_array, report_vocab_length, val_mesh_ids_array, 
#                                                    val_mesh_ids_array_shifted, mesh_vocab_length, batch_size)

In [32]:
new_experiment.run_experiment(train_token_ids_array, one_hot_mesh_shifted_train, one_hot_mesh_train, 
                              val_token_ids_array, one_hot_mesh_shifted_val, one_hot_mesh_val)

Train on 5148 samples, validate on 300 samples
Epoch 1/50
5148/5148 [==============================] - 23s 5ms/sample - loss: 2.4062 - accuracy: 0.5744 - binary_accuracy: 0.9947 - recall: 0.4575 - precision: 0.7400 - val_loss: 1.6555 - val_accuracy: 0.6926 - val_binary_accuracy: 0.9961 - val_recall: 0.5648 - val_precision: 0.9065
Epoch 2/50
5148/5148 [==============================] - 19s 4ms/sample - loss: 1.7693 - accuracy: 0.6328 - binary_accuracy: 0.9959 - recall: 0.5381 - precision: 0.9083 - val_loss: 1.4552 - val_accuracy: 0.7033 - val_binary_accuracy: 0.9965 - val_recall: 0.5583 - val_precision: 0.9948
Epoch 3/50
5148/5148 [==============================] - 19s 4ms/sample - loss: 1.5798 - accuracy: 0.6526 - binary_accuracy: 0.9964 - recall: 0.5706 - precision: 0.9431 - val_loss: 1.2632 - val_accuracy: 0.7154 - val_binary_accuracy: 0.9970 - val_recall: 0.6613 - val_precision: 0.9357
Epoch 4/50
5148/5148 [==============================] - 20s 4ms/sample - loss: 1.4294 - accuracy: 

In [68]:
new_experiment.run_experiment(one_hot_reports_train, one_hot_mesh_shifted_train, one_hot_mesh_train, 
                              one_hot_reports_val, one_hot_mesh_shifted_val, one_hot_mesh_val)

Train on 5148 samples, validate on 300 samples
Epoch 1/50
5148/5148 [==============================] - 8s 2ms/step - loss: 2.5586 - acc: 0.5653 - binary_accuracy: 0.9943 - recall: 0.4254 - precision: 0.6532 - val_loss: 1.7772 - val_acc: 0.6921 - val_binary_accuracy: 0.9956 - val_recall: 0.5446 - val_precision: 0.8309
Epoch 2/50
5148/5148 [==============================] - 6s 1ms/step - loss: 1.9180 - acc: 0.6210 - binary_accuracy: 0.9955 - recall: 0.5235 - precision: 0.8424 - val_loss: 1.5358 - val_acc: 0.6915 - val_binary_accuracy: 0.9966 - val_recall: 0.5628 - val_precision: 0.9959
Epoch 3/50
5148/5148 [==============================] - 6s 1ms/step - loss: 1.7078 - acc: 0.6320 - binary_accuracy: 0.9961 - recall: 0.5258 - precision: 0.9584 - val_loss: 1.3822 - val_acc: 0.7018 - val_binary_accuracy: 0.9967 - val_recall: 0.5808 - val_precision: 0.9882
Epoch 4/50
5148/5148 [==============================] - 7s 1ms/step - loss: 1.5519 - acc: 0.6579 - binary_accuracy: 0.9964 - recall: 0.55

Epoch 31/50
5148/5148 [==============================] - 6s 1ms/step - loss: 0.3486 - acc: 0.9064 - binary_accuracy: 0.9986 - recall: 0.8458 - precision: 0.9704 - val_loss: 0.6973 - val_acc: 0.8323 - val_binary_accuracy: 0.9978 - val_recall: 0.7941 - val_precision: 0.9168
Epoch 32/50
5148/5148 [==============================] - 6s 1ms/step - loss: 0.3198 - acc: 0.9153 - binary_accuracy: 0.9987 - recall: 0.8604 - precision: 0.9721 - val_loss: 0.6936 - val_acc: 0.8362 - val_binary_accuracy: 0.9979 - val_recall: 0.7992 - val_precision: 0.9165
Epoch 33/50
5148/5148 [==============================] - 6s 1ms/step - loss: 0.2904 - acc: 0.9249 - binary_accuracy: 0.9988 - recall: 0.8731 - precision: 0.9755 - val_loss: 0.6939 - val_acc: 0.8372 - val_binary_accuracy: 0.9978 - val_recall: 0.8023 - val_precision: 0.9082
Epoch 34/50
5148/5148 [==============================] - 6s 1ms/step - loss: 0.3115 - acc: 0.9186 - binary_accuracy: 0.9987 - recall: 0.8673 - precision: 0.9684 - val_loss: 0.7553 -

In [47]:
new_experiment.save_weights_history(model_output_dir+'emb_')

## Load results of specific experiment

In [46]:
model_output_dir = dir + 'trained_models/seq2seq/'

In [50]:
epochs = 50
hidden_dim = 512

param_fn = 'emb_param_seq2seq_epochs_{}_hiddendim_{}.pkl'\
.format(epochs, hidden_dim)
params = pickle.load(open(model_output_dir + param_fn, 'rb'))

old_experiment = Seq2Seq(**params)
old_experiment.build_model()
old_experiment.load_weights_history(model_output_dir+'emb_')

In [51]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

In [52]:
def strip_start_end(seq, start_token='start', end_token='end'):
    stripped_seq = []
    for s in seq:
        if s not in [start_token, end_token]:
            stripped_seq.append(s)
    return stripped_seq

In [53]:
for _ in range(10):
    sample = val_df.sample(1)
    true_mesh_caption = list(sample.single_mesh)[0]
    sample_report = list(sample.pad_text_report)[0]
    
    sample_report_ids = []
    for token in sample_report:
        if token in word_to_id.keys():
            sample_report_ids.append(word_to_id[token])
        else:
            sample_report_ids.append(word_to_id[unknown_token])
    
    sample_report_ids = np.array(sample_report_ids).reshape(1, len(sample_report_ids))
    #one_hot_sample_report = dpt.one_hot_sequence(sample_report_ids, report_vocab_length)
    
    #target = predict_sequence(infenc, infdec, one_hot_sample_report, n_steps_out, n_features_out)
    #target = old_experiment.predict_sequence(one_hot_sample_report)
    target = old_experiment.predict_sequence(sample_report_ids)
    predicted_mesh_ids = one_hot_decode(target)
    predicted_mesh = [id_to_mesh[idx] for idx in predicted_mesh_ids]
    
    sample_report = strip_start_end(sample_report)
    predicted_mesh = strip_start_end(predicted_mesh)
    
    print('')
    print('Original report: ', sample_report)
    print('True mesh caption: ', true_mesh_caption)
    print('Predicted mesh caption: ', predicted_mesh)


Original report:  ['airspace', 'opacity', 'left', 'upper', 'lung', '.', 'heart', 'size', 'within', 'normal', 'limits', '.', 'mild', 'calcification', 'aortic', '.', 'airspace', 'opacity', 'left', 'upper', 'lung', 'may', 'represent', 'streaky', 'atelectasis', 'resolving', 'pneumonia']
True mesh caption:  ['opacity', 'lung', 'upper_lobe', 'left']
Predicted mesh caption:  ['atherosclerosis', 'aorta', 'opacity', 'lung', 'upper_lobe', 'right', 'opacity', 'lung', 'upper_lobe', 'right']

Original report:  ['heart', 'size', 'within', 'normal', 'limits', '.', 'calcified', 'right', 'hilar', 'lymph', 'noted']
True mesh caption:  ['calcinosis', 'lung', 'hilum', 'lymph_nodes', 'right']
Predicted mesh caption:  ['calcinosis', 'lung', 'hilum', 'lymph_nodes', 'right']

Original report:  ['low', 'lung', 'volumes', 'noted', '.', 'allowing', 'technical', 'factors', 'heart', 'size', 'normal', '.', 'mediastinum', 'unremarkable', '.', 'increased', 'bilateral', 'predominantly', 'perihilar', 'interstitial', '

## Evaluate BLEU scores on all trian/val/test data

In [55]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

def evaluate_model(model, df, report_vocab_length):
    actual, predicted = list(), list()
    bleu1, bleu2, bleu3, bleu4 = list(), list(), list(), list()

    for _, sample in df.iterrows():
        true_mesh_caption = sample.single_mesh
        sample_report = sample.pad_text_report

        sample_report_ids = []
        for token in sample_report:
            if token in word_to_id.keys():
                sample_report_ids.append(word_to_id[token])
            else:
                sample_report_ids.append(word_to_id[unknown_token])

        sample_report_ids = np.array(sample_report_ids).reshape(1, len(sample_report_ids))
        #one_hot_sample_report = dpt.one_hot_sequence(sample_report_ids, report_vocab_length)

        #target = predict_sequence(infenc, infdec, one_hot_sample_report, n_steps_out, n_features_out)
        #target = model.predict_sequence(one_hot_sample_report)
        target = model.predict_sequence(sample_report_ids)
        predicted_mesh_ids = one_hot_decode(target)
        predicted_mesh = [id_to_mesh[idx] for idx in predicted_mesh_ids]

        # sample_report = strip_start_end(sample_report)
        yhat = strip_start_end(predicted_mesh)
        reference = true_mesh_caption
        
        # calculate BLEU score
        bleu1.append(sentence_bleu([reference], yhat, weights=(1.0, 0, 0, 0)))
        bleu2.append(sentence_bleu([reference], yhat, weights=(0.5, 0.5, 0, 0)))
        bleu3.append(sentence_bleu([reference], yhat, weights=(0.3, 0.3, 0.3, 0)))
        bleu4.append(sentence_bleu([reference], yhat, weights=(0.25, 0.25, 0.25, 0.25)))
    
        # store actual and predicted
        actual.append(reference)
        predicted.append(yhat)
        
    print('BLEU1: ', np.mean(bleu1)*100)
    print('BLEU2: ', np.mean(bleu2)*100)
    print('BLEU3: ', np.mean(bleu3)*100)
    print('BLEU4: ', np.mean(bleu4)*100)
    
    return actual, predicted

In [56]:
train_actual, train_predicted = evaluate_model(old_experiment, train_df, report_vocab_length)

/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Con

BLEU1:  71.41680682785245
BLEU2:  44.64865037160584
BLEU3:  31.979927161300047
BLEU4:  17.600822157542577


In [90]:
train_actual, train_predicted = evaluate_model(old_experiment, train_df, report_vocab_length)

BLEU1:  69.22569916540729
BLEU2:  41.01887071533283
BLEU3:  28.331983321743117
BLEU4:  15.250252448376528


In [57]:
val_actual, val_predicted = evaluate_model(old_experiment, val_df, report_vocab_length)

BLEU1:  60.06780874781047
BLEU2:  19.520045826637116
BLEU3:  10.345706465515486
BLEU4:  4.569959657318097


In [89]:
val_actual, val_predicted = evaluate_model(old_experiment, val_df, report_vocab_length)

/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Con

BLEU1:  61.21837222560159
BLEU2:  19.801445506807212
BLEU3:  10.961680790378018
BLEU4:  3.9947152852943764


## Evaluate ROUGE scores on all train/val/test data

In [81]:
import rouge

evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                       max_n=4,
                       limit_length=True,
                       length_limit=100,
                       length_limit_type='words',
                       apply_avg='Avg',
                       apply_best='Best',
                       alpha=0.5, # Default F1_score
                       weight_factor=1.2,
                       stemming=True)

In [87]:
def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

In [91]:
train_hypotheses = [' '.join(p) for p in train_predicted]
train_references = [' '.join(a) for a in train_actual]

scores = evaluator.get_scores(train_hypotheses, train_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))

	rouge-1:	P: 69.24	R: 93.56	F1: 76.22
	rouge-2:	P: 41.10	R: 67.14	F1: 47.69
	rouge-3:	P: 29.48	R: 54.22	F1: 34.92
	rouge-4:	P: 18.27	R: 39.25	F1: 22.28
	rouge-l:	P: 72.17	R: 93.74	F1: 79.02
	rouge-w:	P: 68.90	R: 76.60	F1: 70.04


In [92]:
val_hypotheses = [' '.join(p) for p in val_predicted]
val_references = [' '.join(a) for a in val_actual]

scores = evaluator.get_scores(val_hypotheses, val_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))

	rouge-1:	P: 62.76	R: 75.75	F1: 66.47
	rouge-2:	P: 21.96	R: 32.11	F1: 24.45
	rouge-3:	P: 12.68	R: 20.02	F1: 14.62
	rouge-4:	P:  6.89	R: 11.69	F1:  7.96
	rouge-l:	P: 65.17	R: 76.90	F1: 68.87
	rouge-w:	P: 62.21	R: 65.42	F1: 62.24
